<a href="https://colab.research.google.com/github/BYU-Handwriting-Lab/GettingStarted/blob/master/notebooks/HWRActiveTransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active Transfer Learning for Handwriting Recognition

This notebook contains code to combine active and transfer learning into a
cohesive framework. The hope is to allow for new handwriting datasets to be
fitted quickly.

In this notebook, we will start the process of fitting the Washington dataset
when our list of pre-trained models were trained on the IAM, Rimes, and Bentham
datasets.

## Dependencies

First, let's import our dependencies such as Tensorflow, numpy, etc.

In [1]:
import os

import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.constraints as kc

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import editdistance as ed

## Download Datasets and Pre-trained Models

In [3]:
# IAM HWR LINE LEVEL MODEL
# ID: 1LmI0kZLGCwE3uD34AbTsmOilvRaWLoIW
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1LmI0kZLGCwE3uD34AbTsmOilvRaWLoIW' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1LmI0kZLGCwE3uD34AbTsmOilvRaWLoIW" -O iam_hwr_line_level_model.zip && rm -rf /tmp/cookies.txt
!unzip iam_hwr_line_level_model.zip
!rm iam_hwr_line_level_model.zip

# RIMES HWR LINE LEVEL MODEL
# ID: 1MJgNMILauGNfw8fQpEj00pJc6iPDG1uE
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJgNMILauGNfw8fQpEj00pJc6iPDG1uE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MJgNMILauGNfw8fQpEj00pJc6iPDG1uE" -O rimes_hwr_line_level_model.zip && rm -rf /tmp/cookies.txt
!unzip rimes_hwr_line_level_model.zip
!rm rimes_hwr_line_level_model.zip

# BENTHAM HWR LINE LEVEL MODEL
# ID: 1rBCjaLGX0kXjeF4Gwg6JjE3PC5Z_QDq2
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rBCjaLGX0kXjeF4Gwg6JjE3PC5Z_QDq2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1rBCjaLGX0kXjeF4Gwg6JjE3PC5Z_QDq2" -O bentham_hwr_line_level_model.zip && rm -rf /tmp/cookies.txt
!unzip bentham_hwr_line_level_model.zip
!rm bentham_hwr_line_level_model.zip

# WASHINGTON HWR LINE LEVEL MODEL
# ID: 10ia9GA1j0KY48CvQRUds2c-nbi1AeQAm
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10ia9GA1j0KY48CvQRUds2c-nbi1AeQAm' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10ia9GA1j0KY48CvQRUds2c-nbi1AeQAm" -O washington_hwr_line_level_model.zip && rm -rf /tmp/cookies.txt
!unzip washington_hwr_line_level_model.zip
!rm washington_hwr_line_level_model.zip


# IAM HWR LINE LEVEL DATASET
# ID: 1ProJyrkE9NhflpYG0nQTfp-ZSvKLyVS1
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ProJyrkE9NhflpYG0nQTfp-ZSvKLyVS1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ProJyrkE9NhflpYG0nQTfp-ZSvKLyVS1" -O iam_hwr_line_level.zip && rm -rf /tmp/cookies.txt
!unzip iam_hwr_line_level.zip
!rm iam_hwr_line_level.zip

# RIMES HWR LINE LEVEL DATASET
# ID: 1PdomNgeuaTgLG2-8TZqs5clc_sGx4vWq
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1PdomNgeuaTgLG2-8TZqs5clc_sGx4vWq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1PdomNgeuaTgLG2-8TZqs5clc_sGx4vWq" -O rimes_hwr_line_level.zip && rm -rf /tmp/cookies.txt
!unzip rimes_hwr_line_level.zip
!rm rimes_hwr_line_level.zip

# BENTHAM HWR LINE LEVEL DATASET
# ID: 1rFyNkYCkaMG9Mo3d8AFXaLe1fh6Hm3nF
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rFyNkYCkaMG9Mo3d8AFXaLe1fh6Hm3nF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1rFyNkYCkaMG9Mo3d8AFXaLe1fh6Hm3nF" -O bentham_hwr_line_level.zip && rm -rf /tmp/cookies.txt
!unzip bentham_hwr_line_level.zip
!rm bentham_hwr_line_level.zip

# WASHINGTON HWR LINE LEVEL DATASET
# ID: 10WLliclnhq3NnEjrHaNcGvrAnrafg94T
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10WLliclnhq3NnEjrHaNcGvrAnrafg94T' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10WLliclnhq3NnEjrHaNcGvrAnrafg94T" -O washington_hwr_line_level.zip && rm -rf /tmp/cookies.txt
!unzip washington_hwr_line_level.zip
!rm washington_hwr_line_level.zip

# Cleanup
!rm -r __MACOSX

print('Done.')

Streaming output truncated to the last 5000 lines.
  inflating: bentham_hwr_line_level/images/072_168_003_04_01.png  
  inflating: __MACOSX/bentham_hwr_line_level/images/._072_168_003_04_01.png  
  inflating: bentham_hwr_line_level/images/115_011_001_02_34.png  
  inflating: __MACOSX/bentham_hwr_line_level/images/._115_011_001_02_34.png  
  inflating: bentham_hwr_line_level/images/116_627_002_01_01.png  
  inflating: __MACOSX/bentham_hwr_line_level/images/._116_627_002_01_01.png  
  inflating: bentham_hwr_line_level/images/073_074_001_02_01.png  
  inflating: __MACOSX/bentham_hwr_line_level/images/._073_074_001_02_01.png  
  inflating: bentham_hwr_line_level/images/071_182_001_05_12.png  
  inflating: __MACOSX/bentham_hwr_line_level/images/._071_182_001_05_12.png  
  inflating: bentham_hwr_line_level/images/096_034_004_01_05.png  
  inflating: __MACOSX/bentham_hwr_line_level/images/._096_034_004_01_05.png  
  inflating: bentham_hwr_line_level/images/115_065_004_02_11.png  
  inflating:

In [4]:
!ls

bentham_hwr_line_level	      rimes_hwr_line_level_model
bentham_hwr_line_level_model  sample_data
iam_hwr_line_level	      washington_hwr_line_level
iam_hwr_line_level_model      washington_hwr_line_level_model
rimes_hwr_line_level


## Load the data

Here are a few functions to help us load our data into a format compatible with
Tensorflow

In [2]:
# The default list of characters used in the recognition model
DEFAULT_CHARS = " !\"#%&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{|}£¤§°²ÀÉàâçèéêëîôùûœſ€⊥𐌰𐌳𐌴𐌵𐌸𐌺𐌻𐌼𐌾𐍆𐍈"
# The default list of non-punctuation characters needed for the word beam search decoding algorithm
DEFAULT_NON_PUNCTUATION = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz£¤§°²ÀÉàâçèéêëîôùûœſ€⊥𐌰𐌳𐌴𐌵𐌸𐌺𐌻𐌼𐌾𐍆"
# The default list of punctuation characters needed for hte word beam search decoding algorithm
DEFAULT_PUNCTUATION = " !\"#%&'()*+,-./0123456789:;<=>?[]_{|}"

In [3]:
def str_charset_to_lists(charset):
    """
    Turns string containing all desired characters into list of chars and indices. This is required for mapping
    between integer and char representations for use in the recognition model.

    :param charset: charset as string of chars to be represented in model.
    """
    chars = list(charset)
    indices = list(range(1, len(chars) + 1))
    return chars, indices


def get_char2idx(charset):
    """
    A tensorflow lookup table is created and returned which allows us to encode word transcriptions on the fly
    in the tf.data api. A standard python dictionary won't work when tensorflow is running in graph mode. This
    function will return a lookup table to convert between chars and indices.

    :param charset: string containing all desired characters to be represented
    :return: A tensorflow lookup table to convert characters to integers
    """
    chars, indices = str_charset_to_lists(charset)

    char2idx = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            keys=tf.constant(chars, dtype=tf.string),
            values=tf.constant(indices, dtype=tf.int64),
            key_dtype=tf.string,
            value_dtype=tf.int64
        ),
        default_value=0,
        name='char2idx_lookup'
    )

    return char2idx


def get_idx2char(charset):
    """
    A tensorflow lookup table is created and returned which allows us to encode word transcriptions on the fly
    in the tf.data api. A standard python dictionary won't work when tensorflow is running in graph mode. This
    function will return a lookup table to convert between indices and chars.

    :param charset: string containing all desired characters to be represented.
    :return: A tensorflow lookup table to convert integers to characters
    """
    chars, indices = str_charset_to_lists(charset)

    idx2char = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            keys=tf.constant(indices, dtype=tf.int64),
            values=tf.constant(chars, dtype=tf.string),
            key_dtype=tf.int64,
            value_dtype=tf.string
        ),
        default_value='',
        name='idx2char_lookup'
    )

    return idx2char


def pad_or_truncate(t, sequence_size=128):
    """
    Pad or truncate a tensor to a fixed sequence length. Works for use in the tf.data api in graph mode.

    :param t: The tensor to pad or truncate
    :param sequence_size: The final sequence length of the tensor
    :return:
    """
    dim = tf.size(t)
    return tf.cond(tf.equal(dim, sequence_size), lambda: t,
                   lambda: tf.cond(tf.greater(dim, sequence_size), lambda: tf.slice(t, [0], [sequence_size]),
                                   lambda: tf.concat([t, tf.zeros(sequence_size - dim, dtype=tf.int64)], 0)))


def merge_repeating_values(t):
    """
    Merge repeating indices/characters in a tensor. Utilizes only tf.* functions which makes it
    usable in graph mode.

    :param t: The tensor to have repeated indices/characters merged
    :return: A new tensor with repeating values merged
    """
    t2 = tf.roll(tf.pad(t, [[0, 1]], constant_values=-1), -1, 0)[:tf.size(t)]
    not_equal = tf.math.not_equal(t, t2)
    indices = tf.where(not_equal)
    return tf.reshape(tf.gather(t, indices), [-1])


def str_to_idxs(string, char2idx, sequence_size):
    """
    Perform the actual lookup to convert a string to its integer representation. This function also performs
    padding according to the given sequence size. Works for use in the tf.data api in graph mode.

    :param string: The string to be converted
    :param char2idx: The tf lookup table
    :param sequence_size: The final sequence length
    :return: The converted string now in its integer representation
    """
    idxs = tf.map_fn(lambda char: char2idx.lookup(char), tf.strings.unicode_split(string, 'UTF-8'), dtype=tf.int64)
    return pad_or_truncate(idxs, sequence_size=sequence_size)


def idxs_to_str(idxs, idx2char, merge_repeated=True):
    """
    Perform the actual lookup to convert an integer to its string representation.
    Works for use in the tf.data api in graph mode.

    :param idxs: The idxs to be converted
    :param idx2char: The tf lookup table
    :param merge_repeated: Bool indicating whether or not to merge repeating values in the idx tensor
    :return: The converted idxs now in its string representation
    """
    if merge_repeated:
        idxs = merge_repeating_values(idxs)

    string = tf.map_fn(lambda idx: idx2char.lookup(idx), idxs, dtype=tf.string)
    string = tf.strings.reduce_join(string)
    return tf.strings.strip(string)


def str_to_idxs_batch(batch, char2idx, sequence_size=128):
    """
    Perform the same function as str_to_idxs, except a batch of strings are given as input

    :param batch: A batch of strings as tensor, list, or numpy array
    :param char2idx: The tf lookup table
    :param sequence_size: The final sequence length of each string
    :return: The converted strings now in its integer representation
    """
    return tf.map_fn(lambda string: str_to_idxs(string, char2idx, sequence_size=sequence_size), batch,
                     dtype=tf.int64)


def idxs_to_str_batch(batch, idx2char, merge_repeated=True):
    """
    Perform the same function as idxs_to_str, except a batch of idxs are given as input

    :param batch: A batch of idxs as tensor, list, or numpy array
    :param idx2char: The tf lookup table
    :param merge_repeated: Bool indicating whether or not to merge repeating values in the idx tensor
    :return: The converted idxs now in its string representation
    """
    return tf.map_fn(lambda idxs: idxs_to_str(idxs, idx2char, merge_repeated=merge_repeated), batch,
                     dtype=tf.string)


def img_resize_with_pad(img_tensor, desired_size, pad_value=255):
    """
    The standard tf.image.resize_with_pad function does not allow for specifying the pad value,
    so we create a function with that capability here. Aspect ratio will be preserved.

    :param img_tensor: The image tensor to be resized and padded
    :param desired_size: The desired size (height, width)
    :param pad_value: The value to pad the tensor with
    """
    img_size = tf.shape(img_tensor)

    img_ratio = img_size[0] / img_size[1]
    desired_ratio = desired_size[0] / desired_size[1]

    if img_ratio >= desired_ratio:
        # Solve by height
        new_height = desired_size[0]
        new_width = int(desired_size[0] // img_ratio)
    else:
        new_height = int(desired_size[1] * img_ratio)
        new_width = desired_size[1]
        # Solve by width

    resized_img = tf.image.resize(img_tensor, (new_height, new_width), method=tf.image.ResizeMethod.BICUBIC)

    pad_height = desired_size[0] - new_height
    pad_width = desired_size[1] - new_width

    img_padded = tf.pad(resized_img, [[pad_height, 0], [0, pad_width], [0, 0]], constant_values=pad_value)

    return img_padded


def read_and_encode_image(img_path, img_size=(64, 1024)):
    """
    Used by both encode_img_and_transcription (training) and encode_img_with_name (inference). This method
    simply loads the image given a file path and performs the necessary encoding/resizing/transposing that
    is necessary for use on the recognition model.

    :param img_path: The path to the desired image
    :param img_size: The size of the image after resizing/padding
    :return: The encoded image in its tensor/integer representation
    """
    img_bytes = tf.io.read_file(img_path)
    img = tf.image.decode_image(img_bytes, channels=1, expand_animations=False)
    img = img_resize_with_pad(img, img_size)
    img = tf.image.per_image_standardization(img)

    return img


def encode_img_and_transcription(img_path, transcription, char2idx, sequence_size=128, img_size: tuple = (64, 1024)):
    """
    The actual function to map image paths and string transcriptions to its tensor/integer representation.

    :param img_path: The path to the desired image
    :param transcription: The transcription of the image in integer form
    :param char2idx: The tf lookup table
    :param sequence_size: The final sequence length for transcriptions
    :param img_size: The size of the image after resizing/padding
    :return: The image and transcription in their tensor/integer representations.
    """
    img = read_and_encode_image(img_path, img_size=img_size)
    line = str_to_idxs(transcription, char2idx, sequence_size)
    return img, line


def encode_img_with_name(img_path, img_size=(64, 1024)):
    """
    Used to map img_paths to encoded images for inference. Returned is the encoded image and image name.

    :param img_path: The file path to the image
    :param img_size: The size of the image after resizing/padding
    :return: The encoded image and image path
    """
    img = read_and_encode_image(img_path, img_size)
    return img, img_path


def get_dataset_size(csv_path):
    """
    The tf.data api has a hard time producing the the dataset size. The cardinality() method often
    returns unknown even with the CsvDataset. This function uses pandas to get the length.

    :param csv_path: The path to csv containing information about the dataset
    :return: The size of the dataset
    """
    return len(pd.read_csv(csv_path, sep='\t', header=None, names=['img_path', 'transcription']))


def get_encoded_dataset_from_csv(csv_path, char2idx, max_seq_size, img_size):
    """
    Using the tf.data api, load the desired csv with img_path and transcription data, encode the images and
    transcriptions for use on the recognition model and return the desired tf dataset.

    :param csv_path: The path to the tab delimited csv file containing | Image Path | Transcription |
    :param char2idx: The tf lookup table to map characters to their respective integer representation
    :param max_seq_size: The final sequence length for transcriptions
    :param img_size: The size of the image after resizing/padding (height, width).
    :return: The tf dataset containing encoded images and their respective transcriptions
    """
    path_sep = os.path.sep
    path_prefix = tf.strings.join(csv_path.split('/')[:-1], path_sep)
    return tf.data.experimental.CsvDataset(csv_path, ['img', 'trans'], field_delim='\t', use_quote_delim=False).map(
        lambda img_path, transcription: encode_img_and_transcription(
            tf.strings.join([path_prefix, tf.strings.reduce_join(tf.strings.split(img_path, '/'), separator=path_sep)],
                            separator=path_sep),
            transcription, char2idx, max_seq_size, img_size),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)


def get_encoded_inference_dataset_from_img_path(img_path, img_size):
    """
    Using the tf.data api, load all images from the desired path and return a dataset containing encoded images
    and the image name (without path or extension information).

    :param img_path: The path to the directory containing images
    :param img_size: The size of the image after resizing/padding (height, width)
    :return: The tf dataset containing encoded images and their respective string names
    """
    return tf.data.Dataset.list_files(img_path + '/*', shuffle=False).map(
        lambda path: encode_img_with_name(path, img_size),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Build the Model

Here is the necessary code we need to build the model for training

In [4]:
class FullGatedConv2D(kl.Conv2D):
    """
    Gated Convolutional Layer as described in the paper, Gated Convolutional Recurrent Neural Networks for Multilingual
    Handwriting Recognition (https://ieeexplore-ieee-org.erl.lib.byu.edu/document/8270042). Code obtained from the Flor
    implementation (https://github.com/arthurflor23/handwritten-text-recognition)
    """

    def __init__(self, filters, **kwargs):
        """
        :param filters: The number of filters to be used for the convolution
        :param kwargs: Additional kwargs to be passed to Conv2D
        """
        super(FullGatedConv2D, self).__init__(filters=filters * 2, **kwargs)
        self.nb_filters = filters

    def call(self, inputs):
        """
        Forward pass of gated convolutional layer

        :param inputs: The input to the gated convolution as tensor
        :return: The output of the gated convolution as tensor
        """
        output = super(FullGatedConv2D, self).call(inputs)
        linear = kl.Activation("linear")(output[:, :, :, :self.nb_filters])
        sigmoid = kl.Activation("sigmoid")(output[:, :, :, self.nb_filters:])
        return kl.Multiply()([linear, sigmoid])

    def compute_output_shape(self, input_shape):
        output_shape = super(FullGatedConv2D, self).compute_output_shape(input_shape)
        return tuple(output_shape[:3]) + (self.nb_filters,)

    def get_config(self):
        config = super(FullGatedConv2D, self).get_config()
        config['nb_filters'] = self.nb_filters
        del config['filters']
        return config

In [5]:
class FlorRecognizer(tf.keras.Model):
    """
    Handwriting Recognition Model as described in the Blog Post,
    https://medium.com/@arthurflor23/handwritten-text-recognition-using-tensorflow-2-0-f4352b7afe16.
    This model combines ideas from the following papers:
    - https://ieeexplore-ieee-org.erl.lib.byu.edu/document/8270042
    - http://www.jpuigcerver.net/pubs/jpuigcerver_icdar2017.pdf
    """
    def __init__(self, vocabulary_size=197):
        """
        Define the model in terms of Keras layers

        :param vocabulary_size: The number of possible classes that a character could belong to
        """
        super(FlorRecognizer, self).__init__(name='flor_recognizer')

        self.permute = kl.Permute([2, 1, 3])
        self.conv1 = tf.keras.Sequential(name='conv1')
        self.conv1.add(
            kl.Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), padding="same", kernel_initializer="he_uniform"))
        self.conv1.add(kl.PReLU(shared_axes=[1, 2]))
        self.conv1.add(kl.BatchNormalization(renorm=True))
        self.conv1.add(FullGatedConv2D(filters=16, kernel_size=(3, 3), padding="same"))

        self.conv2 = tf.keras.Sequential(name='conv2')
        self.conv2.add(
            kl.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer="he_uniform"))
        self.conv2.add(kl.PReLU(shared_axes=[1, 2]))
        self.conv2.add(kl.BatchNormalization(renorm=True))
        self.conv2.add(FullGatedConv2D(filters=32, kernel_size=(3, 3), padding="same"))

        self.conv3 = tf.keras.Sequential(name='conv3')
        self.conv3.add(
            kl.Conv2D(filters=64, kernel_size=(2, 4), strides=(2, 4), padding="same", kernel_initializer="he_uniform"))
        self.conv3.add(kl.PReLU(shared_axes=[1, 2]))
        self.conv3.add(kl.BatchNormalization(renorm=True))
        self.conv3.add(
            FullGatedConv2D(filters=64, kernel_size=(3, 3), padding="same", kernel_constraint=kc.MaxNorm(4, [0, 1, 2])))
        self.dropout1 = kl.Dropout(rate=0.2, name='dropout1')

        self.conv4 = tf.keras.Sequential(name='conv4')
        self.conv4.add(
            kl.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer="he_uniform"))
        self.conv4.add(kl.PReLU(shared_axes=[1, 2]))
        self.conv4.add(kl.BatchNormalization(renorm=True))
        self.conv4.add(
            FullGatedConv2D(filters=128, kernel_size=(3, 3), padding="same", kernel_constraint=kc.MaxNorm(4, [0, 1, 2])))
        self.dropout2 = kl.Dropout(rate=0.2, name='dropout2')

        self.conv5 = tf.keras.Sequential(name='conv5')
        self.conv5.add(
            kl.Conv2D(filters=256, kernel_size=(2, 4), strides=(2, 4), padding="same", kernel_initializer="he_uniform"))
        self.conv5.add(kl.PReLU(shared_axes=[1, 2]))
        self.conv5.add(kl.BatchNormalization(renorm=True))
        self.conv5.add(
            FullGatedConv2D(filters=256, kernel_size=(3, 3), padding="same", kernel_constraint=kc.MaxNorm(4, [0, 1, 2])))
        self.dropout3 = kl.Dropout(rate=0.2, name='dropout3')

        self.conv6 = tf.keras.Sequential(name='conv6')
        self.conv6.add(
            kl.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer="he_uniform"))
        self.conv6.add(kl.PReLU(shared_axes=[1, 2]))
        self.conv6.add(kl.BatchNormalization(renorm=True))

        self.mp = kl.MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding="valid", name='mp')

        self.gru1 = tf.keras.Sequential(name='gru1')
        self.gru1.add(kl.Bidirectional(kl.GRU(units=256, return_sequences=True, dropout=0.5)))
        self.gru1.add(kl.Dense(units=512))
        self.gru1.add(kl.PReLU())

        self.gru2 = tf.keras.Sequential(name='gru2')
        self.gru2.add(kl.Bidirectional(kl.GRU(units=256, return_sequences=True, dropout=0.5)))
        self.gru2.add(kl.Dense(units=vocabulary_size))

    def call(self, x, training=False, **kwargs):
        """
        Forward pass of the Recognizer. The training parameter is passed to certain layers
        in the model that act differently during training than they do during inference.

        :param x: The input to the recognizer (batch, height, width, channels)
        :param training: T/F - Is the model in training or inference mode?
        :param kwargs: Additional parameters
        :return:
        """
        out = self.permute(x)
        # CNN
        out = self.conv1(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.dropout1(out, training=training)
        out = self.conv4(out)
        out = self.dropout2(out, training=training)
        out = self.conv5(out)
        out = self.dropout3(out, training=training)
        out = self.conv6(out)

        # MaxPool and Reshape
        out = self.mp(out)
        # out = tf.squeeze(out)
        out = tf.reshape(out, (-1, out.shape[1], out.shape[2] * out.shape[3]))

        # RNN
        out = self.gru1(out)
        out = self.gru2(out)

        return out

In [15]:
class ModelTrainer:
    """
    Responsible for training the model. Scope becomes an issue when dealing with @tf.function.
    It's easier to place all of the training code into an object so we don't run into issues.
    Once the object is created, the __call__ method will train and return the results and the trained model.
    """

    def __init__(self, model, epochs, batch_size, train_dataset, train_dataset_size, val_dataset, val_dataset_size, lr=4e-4, max_seq_size=128):
        """
        Set up necessary variables that will be used during training, including the model, optimizer,
        encoder, and other metrics.

        :param model: The recognition model
        :param epochs: Number of epochs to train the model
        :param batch_size: How many images are in a mini-batch during training
        :param train_dataset: Train Dataset that is mapped and batched (see train method for context)
        :param train_dataset_size: The number of images in the training set
        :param lr: The learning rate of the model
        :param max_seq_size: The maximum length of a line-level transcription (See Encoder for context)
        """
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.train_dataset = train_dataset
        self.train_dataset_size = train_dataset_size
        self.val_dataset = val_dataset
        self.val_dataset_size = val_dataset_size
        self.optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
        self.max_seq_size = max_seq_size
        self.train_loss = tf.keras.metrics.Mean(name='train_loss')
        self.val_loss = tf.keras.metrics.Mean(name='val_loss')

    def save_model(self):
        tf.print('Saving Model Weights to', self.model_out)
        self.model.save_weights(self.model_out)

    @tf.function
    def train_step(self, images, labels):
        """
        One training step given a mini-batch of images and labels. Note the use of the annotation, @tf.function.
        This annotation will allow TensorFlow to analyze the method and speed up training. However you must be
        careful on what can go inside a @tf.function. See the following links for details:
        * https://www.tensorflow.org/api_docs/python/tf/function
        * https://www.tensorflow.org/guide/function
        * https://pgaleone.eu/tensorflow/tf.function/2019/03/21/dissecting-tf-function-part-1/
        * https://pgaleone.eu/tensorflow/tf.function/2019/04/03/dissecting-tf-function-part-2/

        :param images: mini-batch of images in tensor format
        :param labels: mini-batch of index labels in tensor format
        """
        with tf.GradientTape() as tape:
            iter_batch_size = images.shape[0]

            # Items needed for CTC-Loss
            input_lengths = tf.constant(np.full((iter_batch_size,), self.max_seq_size))
            label_lengths = tf.math.count_nonzero(labels, axis=1)
            unique_labels = tf.nn.ctc_unique_labels(labels)

            # Make a prediction based on a batch of images
            predictions = self.model(images, training=True)

            # Calculate the loss
            loss = tf.nn.ctc_loss(labels, predictions, label_lengths, input_lengths,
                                  logits_time_major=False, unique=unique_labels)
            loss = tf.reduce_mean(loss)

        # Gradient Tape caught the gradients, now apply the gradients to the model
        # using the optimizer and update our loss metric
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        self.train_loss(loss)
    
    @tf.function
    def validation_step(self, images, labels):
        """
        One validation step given a mini-batch of images and labels. Note the use of the annotation, @tf.function.
        This annotation will allow TensorFlow to analyze the method and speed up training. However you must be
        careful on what can go inside a @tf.function. See the following links for details:
        * https://www.tensorflow.org/api_docs/python/tf/function
        * https://www.tensorflow.org/guide/function
        * https://pgaleone.eu/tensorflow/tf.function/2019/03/21/dissecting-tf-function-part-1/
        * https://pgaleone.eu/tensorflow/tf.function/2019/04/03/dissecting-tf-function-part-2/

        :param images: mini-batch of images in tensor format
        :param labels: mini-batch of index labels in tensor format
        """
        iter_batch_size = images.shape[0]

        # Items needed for CTC-Loss
        input_lengths = tf.constant(np.full((iter_batch_size,), self.max_seq_size))
        label_lengths = tf.math.count_nonzero(labels, axis=1)
        unique_labels = tf.nn.ctc_unique_labels(labels)

        # Make a prediction based on a batch of images
        predictions = self.model(images)

        # Calculate the loss and update the metric
        loss = tf.nn.ctc_loss(labels, predictions, label_lengths, input_lengths,
                              logits_time_major=False, unique=unique_labels)
        loss = tf.reduce_mean(loss)
        self.val_loss(loss)

    def train(self):
        """
        Main training loop. The method will training for the specified number of epochs and
        will iterate through the training and validation sets each epoch. The training and
        validation loss will be printed to the screen with a progress bar.

        :return: The model and losses during training
        """
        train_losses = []
        val_losses = []

        # Place in a try/except and return the model/metrics in case we want to stop midway through training
        try:
            # Main loop to go through each dataset for n epochs
            for epoch in range(self.epochs):
                # Reset our metrics for each epoch
                self.train_loss.reset_states()
                self.val_loss.reset_states()

                # Train Loop
                train_loop = tqdm(total=int(np.ceil(self.train_dataset_size / self.batch_size)), position=0, leave=True)
                for images, labels in self.train_dataset:
                    # Take a train step and update our progress bar
                    self.train_step(images, labels)
                    train_loop.set_description('Train - Epoch: {}, Loss: {:.4f}'
                                               .format(epoch, self.train_loss.result()))
                    train_loop.update(1)
                train_loop.close()

                # Validation Loop
                val_loop = tqdm(total=int(np.ceil(self.val_dataset_size / self.batch_size)), position=0, leave=True)
                for images, labels in self.val_dataset:
                    # Take a validation step and update our progress bar
                    self.validation_step(images, labels)
                    val_loop.set_description('Val   - Epoch: {}, Loss: {:.4f}'
                                             .format(epoch, self.val_loss.result()))
                    val_loop.update(1)
                val_loop.close()

                train_losses.append(self.train_loss.result().numpy())
                val_losses.append(self.val_loss.result().numpy())

        except Exception as e:
            print("Error: {0}".format(e))
        finally:
            tf.print('Finished Training')
            return self.model, train_losses

    def __call__(self):
        """
        Calls self.train() method which contains main training loop.

        :return: The model and train/val losses as tuple
        """
        return self.train()

## Additional Helper Functions

In [7]:
def prediction_confidence(output, prediction):
    """
    Given the model output, give a confidence score for the given prediction

    :param output: The model's output, shape: (sequence x num_classes)
    :param prediction: The model's prediction, shape: (sequence)
    :return: The confidence score
    """
    batch_size = output.shape[0]
    seq_size = output.shape[1]

    values = merge_repeating_values(prediction)
    mask = tf.not_equal(values, tf.constant(0, dtype=tf.int64))
    unpadded_label = tf.boolean_mask(values, mask)
    label = tf.expand_dims(pad_or_truncate(unpadded_label, sequence_size=seq_size), 0)

    input_lengths = tf.constant(np.full(batch_size, seq_size))
    label_lengths = tf.math.count_nonzero(label, axis=1)

    loss = tf.nn.ctc_loss(label, output, label_lengths, input_lengths, logits_time_major=False)

    probability = tf.exp(-loss)[0].numpy()

    return probability


def bytes_to_unicode(byte_string_tensor):
    """
    Takes a tensor of byte strings and converts them to a regular python list of unicode strings.
    This is useful when string tensors need to be converted for error rate calculations.

    :param: byte_string_tensor: A tensor of byte strings
    :return: A python list of strings
    """
    return [s.decode('utf8') if type(s) == np.bytes_ or type(s) == bytes else s
            for s in byte_string_tensor.numpy()]


class ErrorRates:
    def __init__(self):
        self.cer_total_error = 0
        self.wer_total_error = 0
        self.count = 0

    def cer(self, y_true, y_pred):
        y_true = ' '.join(y_true.split())
        y_pred = ' '.join(y_pred.split())

        dist = ed.eval(y_true, y_pred)
        if len(y_true) == 0:
            return len(y_pred)

        self.cer_total_error += float(dist) / float(len(y_true))

    def wer(self, y_true, y_pred):
        y_true = y_true.split()
        y_pred = y_pred.split()

        dist = ed.eval(y_true, y_pred)
        if len(y_true) == 0:
            return len(y_pred)

        self.wer_total_error += float(dist) / float(len(y_true))

    def update(self, y_true, y_pred):
        self.cer(y_true, y_pred)
        self.wer(y_true, y_pred)
        self.count += 1

    def get_error_rates(self):
        return (self.cer_total_error / self.count, self.wer_total_error / self.count) if self.count > 0 else (1.0, 1.0)

    def __call__(self, y_true, y_pred):
        self.update(y_true, y_pred)

def evaluate_model(model, dataset, size):
    idx2char = get_idx2char(DEFAULT_CHARS)

    y_pred = []
    y_true = []

    inference_loop = tqdm(total=int(np.ceil(size/32)), position=0, leave=True)
    for img, label in dataset.batch(32):
        output = model(img)
        predictions = tf.argmax(output, 2)
        str_preds = idxs_to_str_batch(predictions, idx2char, merge_repeated=True)
        str_labels = idxs_to_str_batch(label, idx2char, merge_repeated=False)
        y_pred.extend(bytes_to_unicode(str_preds))
        y_true.extend(bytes_to_unicode(str_labels))
        inference_loop.update(1)
    inference_loop.close()

    error_rates = ErrorRates()

    rate_loop = tqdm(total=len(y_pred), position=0, leave=True)
    for pred, label in zip(y_pred, y_true):
        error_rates.update(label, pred)
        rate_loop.update(1)
    rate_loop.close()
    
    return error_rates.get_error_rates()

## Confidence/Masking Functions

In [17]:
def find_closest_non_blank(prediction, i):
    size = len(prediction)
    # Look left:
    left_index = i
    while left_index >= 0 and (prediction[left_index] == 0 or prediction[left_index] == 1):
        left_index -= 1
    left_distance = i - left_index if left_index >= 0 else 200
    
    right_index = i
    while right_index < size and (prediction[right_index] == 0 or prediction[right_index] == 1):
        right_index += 1
    right_distance = right_index - i if right_index < size else 200

    if right_distance < left_distance:
        return right_index
    else:
        return left_index

def fix_label_output(label, idx2char, char2idx, sequence_size=128):
    string = idxs_to_str(label, idx2char, merge_repeated=True).numpy().decode('utf8')
    string = " ".join(string.split())

    label = str_to_idxs(string, char2idx, sequence_size=sequence_size)
    return label


def get_confident_img_label(img, confidence, prediction, idx2char, char2idx, prob_threshold=0.85, timestep_width=1024/128, white=0.2280537):
    new_img = img.numpy()
    new_label = prediction.numpy()
    prev_conf = 0
    for i in range(128):
        if prediction[i] != 0:
            conf = confidence[i]
            prev_conf = conf
        else:
            conf = prev_conf

        if conf < prob_threshold:
            start = round(i * timestep_width)
            end = round((i + 1) * timestep_width)
            new_img[:, start:end] = white
            new_label[i] = 1
    
    return new_img, new_label
            

@tf.function
def model_inference(model, img):
    return model(img)


def create_confident_dataset(model, dataset, dataset_size, idx2char, char2idx, prob_threshold=0.85, dataset_path='./confident_dataset'):
    dataset_file_path = os.path.join(dataset_path, 'confident_train.tsv')
    dataset_data = []
    img_num = 0

    inference_loop = tqdm(total=int(np.ceil(dataset_size/32)), position=0, leave=True)
    for img, label in dataset.shuffle(1000).batch(32):
        output = model_inference(model, img)
        output = tf.nn.softmax(output)
        confidences = tf.reduce_max(output, 2)
        predictions = tf.argmax(output, 2)

        # Create Confident Dataset
        for single_img, single_conf, single_pred in zip(img, confidences, predictions):
            new_img, new_label = get_confident_img_label(single_img, single_conf, single_pred, idx2char, char2idx, prob_threshold=prob_threshold)

            img_path = os.path.join('./images', str(img_num) + 'jpg')
            img_num += 1

            encoded_img = tf.image.encode_jpeg(new_img, quality=100)
            tf.io.write_file(os.path.join(dataset_path, img_path), encoded_img)
            transcription = idxs_to_str(new_label, idx2char, merge_repeated=True)

            dataset_data.append([img_path, transcription])
        inference_loop.update(1)
    inference_loop.close()

    df = pd.DataFrame(dataset_data)
    df.to_csv(dataset_file_path, sep='\t', index=False, header=False)

    return dataset_file_path


def evaluate_and_create_confident_dataset(model, dataset, dataset_size, idx2char, char2idx, prob_threshold=.95):
    y_pred = []
    y_true = []

    new_imgs = []
    new_labels = []

    inference_loop = tqdm(total=int(np.ceil(dataset_size/32)), position=0, leave=True)
    for img, label in dataset.batch(32):
        output = model_inference(model, img)
        predictions = tf.argmax(output, 2)

        # Create Confident Dataset
        for single_img, single_label, single_output in zip(img, label, output):
            new_img, new_label = get_confident_img_label(single_img, single_output, idx2char, char2idx, prob_threshold=prob_threshold)
            new_imgs.append(new_img)
            new_labels.append(new_label)

        # Calculate Error Rates
        str_preds = idxs_to_str_batch(predictions, idx2char, merge_repeated=True)
        str_labels = idxs_to_str_batch(label, idx2char, merge_repeated=False)
        y_pred.extend(bytes_to_unicode(str_preds))
        y_true.extend(bytes_to_unicode(str_labels))
        inference_loop.update(1)
    inference_loop.close()

    error_rates = ErrorRates()

    rate_loop = tqdm(total=len(y_pred), position=0, leave=True)
    for pred, label in zip(y_pred, y_true):
        error_rates.update(label, pred)
        rate_loop.update(1)
    rate_loop.close()

    dataset = tf.data.Dataset.from_tensor_slices((new_imgs, new_labels))
    
    return dataset, error_rates.get_error_rates()

## Experimenting...

In [ ]:
target_dataset_path = 'rimes_hwr_line_level/rimes_train.tsv'
train_dataset_path = 'iam_hwr_line_level/iam_train.tsv'
val_dataset_path = 'rimes_hwr_line_level/rimes_test.tsv'
model_path = 'iam_hwr_line_level_model/run1'
prob_threshold = 0.75

# Self-Training Settings
ST_EPOCHS = 2
ST_BATCH_SIZE = 32
ST_LEARNING_RATE = 0.0004

char2idx = get_char2idx(DEFAULT_CHARS)
idx2char = get_idx2char(DEFAULT_CHARS)

train_dataset_size = get_dataset_size(train_dataset_path)
train_dataset = get_encoded_dataset_from_csv(train_dataset_path, char2idx, 128, (64, 1024))

val_dataset_size = get_dataset_size(val_dataset_path)
val_dataset = get_encoded_dataset_from_csv(val_dataset_path, char2idx, 128, (64, 1024))\
    .batch(ST_BATCH_SIZE)

target_dataset_size = get_dataset_size(target_dataset_path)
target_dataset = get_encoded_dataset_from_csv(target_dataset_path, char2idx, 128, (64, 1024))

model = FlorRecognizer(len(DEFAULT_CHARS) + 1)
model.load_weights(model_path)

while True:
    st_dataset_path = create_confident_dataset(model, target_dataset, target_dataset_size, idx2char, char2idx, prob_threshold=prob_threshold)

    st_dataset_size = get_dataset_size(st_dataset_path)
    st_dataset = get_encoded_dataset_from_csv(st_dataset_path, char2idx, 128, (64, 1024))
    
    full_dataset = tf.data.Dataset.zip((st_dataset, train_dataset))\
        .flat_map(lambda tuple1, tuple2: tf.data.Dataset.from_tensor_slices(([tuple1[0], tuple2[0]], [tuple1[1], tuple2[1]])))\
        .shuffle(1000)\
        .batch(ST_BATCH_SIZE)

    model_trainer = ModelTrainer(model, ST_EPOCHS, ST_BATCH_SIZE, full_dataset, min(st_dataset_size, train_dataset_size) * 2, val_dataset, val_dataset_size, lr=ST_LEARNING_RATE)
    model, losses = model_trainer.train()
    del st_dataset

  0%|          | 0/355 [00:00<?, ?it/s]

100%|██████████| 355/355 [24:09<00:00,  4.08s/it]
Train - Epoch: 0, Loss: 74.5523: : 386it [04:31,  1.42it/s]
Val   - Epoch: 0, Loss: 111.8280: 100%|██████████| 25/25 [00:09<00:00,  2.53it/s]
Train - Epoch: 1, Loss: 61.9941: : 386it [04:20,  1.48it/s]
Val   - Epoch: 1, Loss: 104.2165: 100%|██████████| 25/25 [00:07<00:00,  3.42it/s]

Finished Training



100%|██████████| 355/355 [24:19<00:00,  4.11s/it]
Train - Epoch: 0, Loss: 60.6790: : 386it [04:31,  1.42it/s]
Val   - Epoch: 0, Loss: 105.7096: 100%|██████████| 25/25 [00:09<00:00,  2.62it/s]
Train - Epoch: 1, Loss: 55.4090: : 386it [04:20,  1.48it/s]
Val   - Epoch: 1, Loss: 108.8578: 100%|██████████| 25/25 [00:07<00:00,  3.47it/s]

Finished Training


...


100%|██████████| 355/355 [26:01<00:00,  4.40s/it]
Train - Epoch: 0, Loss: 32.9612: : 386it [04:55,  1.31it/s]
Val   - Epoch: 0, Loss: 109.9605: 100%|██████████| 25/25 [00:09<00:00,  2.52it/s]
Train - Epoch: 1, Loss: 31.3414: : 386it [04:38,  1.39it/s]
Val   - Epoch: 1, Loss: 116.4185: 100%|██████████| 25/25 [00:07<00:00,  3.29it/s]

Finished Training



100%|██████████| 355/355 [25:25<00:00,  4.30s/it]
Train - Epoch: 0, Loss: 33.0408: : 386it [04:49,  1.33it/s]
Val   - Epoch: 0, Loss: 142.0362: 100%|██████████| 25/25 [00:09<00:00,  2.51it/s]
Train - Epoch: 1, Loss: 31.4513: : 386it [04:41,  1.37it/s]
Val   - Epoch: 1, Loss: 121.1604: 100%|██████████| 25/25 [00:07<00:00,  3.21it/s]

Finished Training



Train - Epoch: 0, Loss: 31.4595:  52%|█████▏    | 196/379 [02:28<02:09,  1.42it/s]

## FULL

Out of all of our pre-trained models, let's find the best model to start with.

In [ ]:
# First, load in all of our pre-trained models
iam_model = Recognizer()
iam_model.load_weights()

rimes_model = Recognizer()
rimes_model.load_weights()

bentham_model = Recognizer()
bentham_model.load_weights()

models = [iam_model, rimes_model, bentham_model]

# Take a subset of the dataset to use for testing
dataset = get_inference_dataset_from_...

# Metrics to help us select the best model 
mean_confidence = tf.keras.metrics.Mean(name="confidence")
best_model = None
best_score = 1000

# Iterate over each of the models
for model in models:
    mean_confidence.reset_states()

    # Iterate over the subset of the dataset
    for img, img_name in dataset:
        output = model(img)
        prediction, confidence = predict_with_confidence(output)
    
    if mean_confidence.result() < best_score:
        best_model = model

print('Best Model:', best_model.name)
print('Confidence Score:', best_score)

In [ ]:
# Run through the dataset and see if there are any lines where we are highly
# confident in our predictions
model = best_model
confidence_threshold = .4

training_set = []
labeling_set = []

for img, img_name in dataset:
    output = model(img)
    prediction, confidence = predict_with_confidence(output)

    if confidence > confidence_threshold:
        training_set.append([img, img_name])